Primeira parte:
    • Representação (genótipo): string de bits #OK
    • Recombinação: “cut-and-crossfill” crossover #OK
    • Probabilidade de Recombinação: 90%
    • Mutação: troca de genes #OK
    • Probabilidade de Mutação: 40%
    • Seleção de pais: ranking - Melhor de 2 de 5 escolhidos aleatoriamente #OK
    • Seleção de sobreviventes: substituição do pior
    • Tamanho da população: 100 #OK
    • Número de filhos gerados: 2
    • Inicialização: aleatória #OK
    • Condição de término: Encontrar a solução, ou 10.000 avaliações de fitness #OK
    • Fitness #OK

In [129]:
from random import randint
import numpy as np

In [130]:
populationSize = 100

### Funções auxiliares

In [131]:
def binaryToDecimal(binary): 
    binary1 = binary
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return decimal

In [132]:
def sortPopulation(population):
    # Ordena uma população pelo seu valor de fitness
    population.sort(key=lambda x:x['fitnessValue'])
    return population

### Criar população

In [133]:
def generateChess():
    aux = []
    while(len(aux) != 8):
        value = generateQueen()
        if not value in aux:
            aux.append(value)
    return aux

In [134]:
def generateQueen():
    queenPosition = ''
    for bitPosition in range(1,4):
        queenPosition += str(randint(0,1))
    return queenPosition

In [135]:
def generatePopulation():
    aux = []
    for populationMember in range(populationSize):
        aux.append({'individual': generateChess() , 'fitnessValue': -1 }) 
        #Cria um objeto composto do individuo e seu valor de fitness    
    return aux

### Quantidade de 'choques'

In [136]:
# Se duas rainhas estiverem na mesma coluna ### OK
def columnThreaten(individual):
    threatenColumn = 0
    for column in range(0,8):
        binaryColumn = np.binary_repr(column, width=3)
        if individual.count(binaryColumn) >= 2:
            threatenColumn += 1
    return threatenColumn

In [137]:
# Se duas rainhas estiverem na mesma diagonal
def diagonalThreaten(individual):
    deltaRow = 0
    deltaCol = 0
    threatenDiagonal = 0
    for x in range(0,8):
        for y in range(0,8):
            if x != y:
                deltaRow = abs(x-y)
                deltaCol = abs(binaryToDecimal(int(individual[x])) - binaryToDecimal(int(individual[y])))
                if (deltaRow == deltaCol):
                    threatenDiagonal += 1
    return threatenDiagonal/4

### Calculo do fitness

In [138]:
def fitnessIndividual(individual):
    value = 1 / (1 + (columnThreaten(individual) + diagonalThreaten(individual)))
    return value

In [139]:
def fitness(population):
    fitnessValue = []
    for individual in population:
        individual['fitnessValue'] = fitnessIndividual(individual['individual'])

#### O valor do fitness será dado pela função 1/1+qntdChoques(tabuleiro)
#### Queremos aumentar esse valor de fitness

### Mutação

Escolher um individuo(tabuleiro) aleatório e trocar a posição de uma rainha

In [140]:
def mutationSwap(individual):
    posicaoRainhaEscolhida = randint(0,7) ### A rainha que tera seu valor alterado
    valorNovaRainha = generateQueen() ##
    posicaoRainha2 = individual['individual'].index(valorNovaRainha)
    #### Trocar os valores da posicao da rainhaEscolhida com a Rainha 2
    aux = individual['individual'][posicaoRainhaEscolhida]
    individual['individual'][posicaoRainhaEscolhida] = individual['individual'][posicaoRainha2]
    individual['individual'][posicaoRainha2] = aux
    return individual

### Seleção de pais

In [141]:
def chooseRandomIndividuals(population):
    aux = []
    for i in range(0,5):
        value = randint(0,populationSize-1)
        aux.append(population[value])
    return aux

In [142]:
def selectParents(population):
    parents = chooseRandomIndividuals(population) #Array de objetos onde os elementos são os individuos e o valor do fitness
    parents = sortPopulation(parents) # Ordenando os pais de acordo com o fitnessValue
    return (parents[-2], parents[-1]) #Retornando os dois ultimos elementos da lista que são os com melhores fitness

### Seleção dos sobreviventes

#### Remover os dois menos adaptados da população

In [143]:
def replaceElements(population, filho1, filho2):
    aux = sortPopulation(population)
    aux[0] = filho1
    aux[1] = filho2
    return aux

### Recombinação: “cut-and-crossfill”

In [144]:
def copyFirstParent(parent, point):
    son = []
    for x in range(0,point+1): # Se o primeiro pai for escolhido o filho recebera primeiro os genes dele
            son.append(parent['individual'][x])
    return son

In [145]:
def generateParentTwoAux(parent, point):
    aux = []
    for x in range(point+1, 8):
        aux.append(parent['individual'][x])
    for x in range(0, point+1):
        aux.append(parent['individual'][x])
    return aux

In [146]:
def insertValuesSecondParent(parent, point, son):
    parentCircular = generateParentTwoAux(parent, point)
    while(len(son) != 8):
            for x in parentCircular:
                if not x in son: # Se o valor do 2 pai ainda não existe no filho insere
                    son.append(x)
    return {'individual': son, 'fitnessValue': -1}
        

In [147]:
def recombination(parent_one, parent_two):
    son = []
    point = randint(0,7) # Escolher uma posição para executar
    chooseFirst = randint(0,1)
    if (chooseFirst == 0):
        son = copyFirstParent(parent_one, point) # Se o primeiro pai for escolhido o filho recebera primeiro os genes dele
        return insertValuesSecondParent(parent_two, point, son)
    else:
        son = copyFirstParent(parent_two, point) # Se o segundo pai for escolhido o filho recebera primeiro os genes dele
        return insertValuesSecondParent(parent_one, point, son)

In [148]:
def lookingForFitnessMax(population):
    for individual in population:
        if individual['fitnessValue'] >= 1:
            return True

In [149]:
def maxIndividual(population):
    for individual in population:
        if individual['fitnessValue'] >= 1:
            return individual

In [156]:
def principal():
    population = generatePopulation()
    fitness(population)
    i = 0
    while (i < 1000) :
        print('iteracao: ', i)
        if lookingForFitnessMax(population) == True:
            break
        pai1, pai2 = selectParents(population)
        filho1 = recombination(pai1,pai2)
        filho2 = recombination(pai1,pai2)
        filho1_mutacao = mutationSwap(filho2)
        filho2_mutacao = mutationSwap(filho2)
        fitness(population)
        population = replaceElements(population, filho1_mutacao, filho2_mutacao)
        i += 1
    return maxIndividual(population)

In [166]:
principal()

iteracao:  0
iteracao:  1
iteracao:  2
iteracao:  3
iteracao:  4
iteracao:  5
iteracao:  6
iteracao:  7
iteracao:  8
iteracao:  9
iteracao:  10
iteracao:  11
iteracao:  12
iteracao:  13
iteracao:  14
iteracao:  15
iteracao:  16
iteracao:  17
iteracao:  18
iteracao:  19
iteracao:  20
iteracao:  21
iteracao:  22
iteracao:  23
iteracao:  24
iteracao:  25
iteracao:  26
iteracao:  27
iteracao:  28
iteracao:  29
iteracao:  30
iteracao:  31
iteracao:  32
iteracao:  33
iteracao:  34
iteracao:  35
iteracao:  36
iteracao:  37
iteracao:  38
iteracao:  39
iteracao:  40
iteracao:  41
iteracao:  42
iteracao:  43
iteracao:  44
iteracao:  45
iteracao:  46
iteracao:  47
iteracao:  48
iteracao:  49


{'individual': ['010', '101', '001', '100', '111', '000', '110', '011'],
 'fitnessValue': 1.0}